# Q-learning for FrozenLake 4x4

In this Notebook, we'll implement an agent that plays <b>FrozenLake.</b>

![alt text](http://simoninithomas.com/drlc/Qlearning/frozenlake4x4.png)

The goal of this game is to go from the starting state (S) to the goal state (G) by walking only on frozen tiles (F) and avoid holes (H). There are four directions you can move to, however, the ice is slippery, so you won't always move in the direction you intend to; this is a stochastic environment.

We'll use the implementation of the Frozen Lake environment from OpenAI's gymnasium, where it is described as follows (see also: [link](https://gymnasium.farama.org/environments/toy_text/frozen_lake/))



```
    Winter is here. You and your friends were tossing around a frisbee at the
    park when you made a wild throw that left the frisbee out in the middle of
    the lake. The water is mostly frozen, but there are a few holes where the
    ice has melted. If you step into one of those holes, you'll fall into the
    freezing water. At this time, there's an international frisbee shortage, so
    it's absolutely imperative that you navigate across the lake and retrieve
    the disc. However, the ice is slippery, so you won't always move in the
    direction you intend.
    The surface is described using a grid like the following

        SFFF
        FHFH
        FFFH
        HFFG

    S : starting point, safe
    F : frozen surface, safe
    H : hole, fall to your doom
    G : goal, where the frisbee is located

    The episode ends when you reach the goal or fall in a hole.
    You receive a reward of 1 if you reach the goal, and zero otherwise.
```

Our agent does not know this stochasticity (it needs to discover this), but the environment dynamics are modeled like this: if you choose a direction (i.e., action), you have a 1/3 chance of actually going in that direction, but also a 1/3 chance of taking one of the perpendicular actions. Remember that at the start, an agent doesn't know what an action means. Its policy only outputs a number between [0, 3], and each of those numbers corresponds to something that happens in the environment (here: moving the agent in the grid) and which the agent has to learn to map. In our case, the environment maps those integers to actions like this:

```
  0: LEFT
  1: DOWN
  2: RIGHT
  3: UP
```

To give you an example, if your agent selects action 0, the environment implements action 'LEFT', but since the environment is also stochastic, there is a 33% chance of actually going left, a 33% chance of going up, and a 33% chance of going down. There is 0% chance of going in the reverse direction, in this case 'RIGHT'. If an attempt at moving encounters a wall, there is no movement.

**Once again, your agent does not have this kind of information at the start, I only provide this here to give you some intuition about this specific problem. The agent needs to learn all of this by scratch by probing the environment by taking actions and using the rewards and observations it gets back.**

## Step -1: Install the dependencies on Google Colab

In [ ]:
!pip install gymnasium
!pip install matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.7/163.7 kB 742.6 kB/s eta 0:00:00 kB/s eta 0:00:01:01
  Using cached pillow-10.4.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (9.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 964.1 kB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/320.5 kB ? eta -:--:--

## Step 0: Import the dependencies 📚
We use 2 libraries:
- `Numpy` for our Q-table
- `OpenAI Gymnasium` for our FrozenLake Environment

In [ ]:
import numpy as np
import gymnasium as gym
import matplotlib.pyplot as plt

## Step 1: Create the environment 🎮
Here we'll create the FrozenLake 4x4 environment by simply loading it from Gym.

OpenAI Gym is a library from OpenAI containing many environments that researchers often use to test their reinforcement learning algorithms on.

In [3]:
env = gym.make("FrozenLake-v1", render_mode='rgb_array')

## Step 2: Create the Q-table and initialize it
- To create our initial Q-table, we need to know how much states (rows) and  actions (columns) there are.
- OpenAI Gym provides us a way to do that: `env.action_space.n` and `env.observation_space.n`

In [4]:
action_size = env.action_space.n
print("Amount of actions: ", action_size)
state_size = env.observation_space.n
print("Amount of states: ", state_size)

Amount of actions:  4
Amount of states:  16


In [5]:
# Create our Q-table with state_size rows and action_size columns (16x4)
qtable = np.zeros((state_size, action_size))

print(qtable)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


## Step 3: Set the hyperparameters

In [6]:
total_episodes = 20000       # Total episodes
learning_rate = 0.7          # Learning rate
max_steps = 99               # Max steps per episode
gamma = 0.95                 # Discount rate
# gamma = 1.

# Exploration parameters
exploration = 1.0            # Exploration probability
exploration_min = 0.01       # Minimum exploration probability
exploration_decay = 0.995    # Exponential decay rate for the exploration

## Step 4: The Q-learning algorithm
The Q-learning algorithm goes like this:   
<br>
**Initialize** all Q-values _Q(s,a)_ arbitrarily (often simply **to zero**)    
Repeat **for each episode**:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Reset state _s_ to the starting state  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Repeat **for each step in the episode** until terminal state:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Pick action _a_** for state _s_ according to the current policy (argmax_a over all _Q(s,a)_ values) **or** a random action with small probability  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Execute action _a_ and **observe reward _R_ and next state _s'_**  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Improve _Q(s,a)_** by using the received reward and the maximum Q-value of the next step:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;_Q(s,a)_&nbsp;&nbsp; <---- &nbsp;&nbsp;_Q(s,a) + learning_rate * (R + gamma * max_a'(Q(s',a')) - Q(s,a))_  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Set state _s_ = _s'_ to go to the next step  
<br>


In the next part, we'll go over the implementation of the Q-learning algorithm, and train an agent to navigate itself through the slippery FrozenLake environment.

We'll start off with a completely random agent. This random agent will explore its environment, and learn from its interactions with it. After a while, the agent learns that certain actions in certain states are better than others. The exploration rate decreases over time, so our agent gradually starts using its learned policy more often, until it converges to a policy that on average more or less maximizes the reward in the environment. I stress 'on average', since the environment is very stochastic, and the slippery nature of the floor often makes sure that no matter what the policy of our agent is, it will still end up in a hole in the ice from time to time.

In [7]:
episode_rewards = []
average_episode_rewards = []

# We learn for a predefined amount of episodes:
for episode in range(total_episodes):

    # Reset the environment:
    state, _ = env.reset()
    total_reward = 0

    for step in range(max_steps): # until done = True or it reaches max_steps

        # There is always a chance that we take a random action for exploration:
        if np.random.rand() < exploration:
            action = env.action_space.sample()

        # Else, simply pick the best action according to your current best policy:
        else:
            action = np.argmax(qtable[state])

        # Give this action to your environment, and observe the new state your agent is in and the reward it got:
        new_state, reward, terminated, truncated, _ = env.step(action)

        # Update Q(s,a):= Q(s,a) + learning_rate [R + gamma * max Q(s',a') - Q(s,a)]
        qtable[state, action] = qtable[state, action] + learning_rate * (reward + gamma * np.max(qtable[new_state]) - qtable[state, action])

        # We add the (discounted) reward to the total reward
        total_reward += gamma**step * reward

        # Set our state to the new_state:
        state = new_state

        # If we reached the goal or we fell into a hole:
        if terminated == True:
            break

    # Decay exploration per episode:
    exploration *= exploration_decay
    exploration = max(exploration_min, exploration)

    episode_rewards.append(total_reward)
    average_episode_rewards.append(sum(episode_rewards) / (episode + 1))

    if episode % 1000 == 0:
      print("\nTotal (discounted) reward for episode", episode + 1, ": ", total_reward)
      print("Average total reward over episodes until now: ", average_episode_rewards[-1])
      # Render the last frame of our frozenlake environment:
      plt.imshow(env.render())
      plt.show()


print("\n\nAverage total reward over all episodes: ", sum(episode_rewards) / total_episodes)


Total (discounted) reward for episode 1 :  0.0
Average total reward over episodes until now:  0.0


NameError: name 'plt' is not defined

We can plot the average total reward over all episodes. We see that after a while, the average reward stops improving, meaning we've hit a good policy. To make sure that we've got the best one, we should actually play around with the algorithm's parameters (like the exploration decay etc.), but let's keep it simple and trust that our agent has found the optimal policy.

In [ ]:
import matplotlib.pyplot as plt

iterations = range(0, total_episodes, 1)
plt.plot(iterations, average_episode_rewards)
plt.ylabel('Average total reward over all episodes')
plt.xlabel('Episode')
# plt.ylim(top=250)

The (by now optimal) policy is created by acting greedily (argmax) over the following Q-table

In [ ]:
print("Q-table for the optimal policy:\n")
print(qtable)

The agent doesn't actually know the concept of left, right, up or down. It simply outputs an action, which is an integer between [0, 3]. The environment understands this number, and changes the state according to this action and the environment dynamics (the random slipperyness of the frozen lake). To see what the policy actually is in human language, we'll change it according to the [translation key](https://github.com/openai/gym/blob/master/gym/envs/toy_text/frozen_lake.py) from OpenAI:

```
LEFT = 0
DOWN = 1
RIGHT = 2
UP = 3
```


In [ ]:
translation_key = {0: "LEFT", 1: "DOWN", 2: "RIGHT", 3: "UP"}

policy = np.argmax(qtable, axis=1)
print("Numerical policy:\n\n", np.resize(policy, (4,4)))
human_policy = [translation_key[key] for key in policy]
print("\nHuman policy:\n\n", np.resize(human_policy, (4,4)))

As we can see, for the FrozenLake environment

```
SFFF
FHFH
FFFH
HFFG
```
there are some (at first) seemingly odd policy choices. For example, the first action is always left, instead of e.g. down. This is because the agent learns that if it goes down directly, it might end up somewhere it didn't intend to (slippery floor). If it keeps selecting 'left', however, the only place it will eventually end up is 'down', which is what it really wants.



## Step 5: Use our Q-table to play FrozenLake 👾
After 20 000 episodes, our Q-table converges to an optimal one, which we can use to play Frozen Lake with an optimal policy, thereby on average maximizing the total reward in an episode!  
Let's play an optimal game and really render the actions the agent picks.

In [ ]:
!apt-get install -y xvfb python-opengl ffmpeg
!pip install imageio[ffmpeg]
!pip install pyglet==1.5.11
!pip install pyvirtualdisplay

In [ ]:
!apt-get install -y xvfb > /dev/null 2>&1

from pyvirtualdisplay import Display

# Start virtual display
display = Display(visible=0, size=(1400, 900))
display.start()

In [ ]:
import imageio

frames = []
env = gym.make("FrozenLake-v1", render_mode="rgb_array")
state, _ = env.reset()
frames.append(env.render())
for step in range(max_steps):
  action = np.argmax(qtable[state])
  new_state, reward, terminated, truncated, _ = env.step(action)
  frames.append(env.render())
  if terminated:
    break
  state = new_state

if new_state == 15:
  print("\nReached the goal 🏆")
else:
  print("\nFell into a hole ☠️")

In [ ]:
video_filename = 'cartpole-video.mp4'
imageio.mimsave(video_filename, [np.array(frame) for frame in frames], fps=5)

In [ ]:
from IPython.display import Video

# Display the video
Video(video_filename, embed=True)